In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
from datetime import datetime

In [3]:
sw_lat, sw_long = 38.857489, -94.846741
ne_lat, ne_long = 39.297268, -94.325766
query = f"""
[out:json];
(
  node["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  way["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
  relation["leisure"="golf_course"]({sw_lat}, {sw_long}, {ne_lat}, {ne_long});
);
out center;
"""

response = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
data = response.json()
for el in data["elements"]:
  name = el.get("tags", {}).get("name")
  if name != None:
    print(name)

The Hill Course
The Rock Course
The River Course
Brough Creek National GC
Smiley's Golf Complex
Overland Park Golf Course
Westlinks Golf Course
Swope Memorial Golf Course
Heart of America Golf Course
St Andrews Golf Club
Lake Quivira Country Club
Milburn Country Club
Tomahawk Hills Golf Course
Royal Meadows Golf Club
The Kansas City Country Club
Driving Range
Hodge Park Golf Course
Shoal Creek Golf Course
Cardinal Hill Golf Course
Hillcrest Golf & Country Club
Indian Hills Country Club
Mission Hills Country Club
Blue Hills Country Club
Oakwood Country Club
Minor Park Golf Course
Drumm Farm Golf Club
Winterstone Golf Course
Teetering Rocks Golf Course
Fred Arbanas Golf Course
Par 3 course
The Unity Village Golf Course
Lake of the Forest Golf Course
Shamrock Hills Golf Club
Tiffany Greens Golf Club
Sykes/Lady Overland Park Golf Complex
Staley Farms Golf Club
The Deuce at the National
The Duece at the National
The National Golf Club
Falcon Ridge Golf Course
Country Club of Leawood
Deer Cr

In [80]:
bookateetime_courses = {
  'Shoal Creek': '118-1',
  'Hodge Park': '117-1',
  'WinterStone': '62-1',
  'Adams Pointe': '45-1',
  'Heart of America': '49-1',
  'Sycamore Ridge': '44-1',
  'Paradise Pointe - The Outlaw': '24-1',
  'Paradise Pointe - The Posse': '24-2',
}

chronogolf_courses = {
  'Falcon Lakes': 6633,
}

golfback_courses = {
  'Dubs Dread': '398d44ce-a908-4ce7-8f50-e5f4bdc77b73',
  'Painted Hills': '857a12d4-a9cf-4a43-afe2-60940bdc7438',
  'Drumm Farm - Full': 'd70999c9-d7d4-4008-9f41-4e9551b3c796',
  'Drumm Farm - Executive': '9a1de435-8a46-4840-9cdc-332c3cfea782',
  'Royal Meadows': 'd2278228-4700-4354-95a8-422a8f9a5a16'
}

foreup_courses = {
  'Teetering Rocks': 7341,
  'Heritage Park': 12159,
  'Tomahawk Hills': 11026,
}

loners = {
  'Sunflower Hills': 'https://www.sunflowerhillsgolfcourse.com/TeeTimes',
}

In [81]:
date = '2025-04-26'
players = 4

tee_times_df = pd.DataFrame()

for course in bookateetime_courses.items():
  # Make a GET request to the URL with the course name and date
  response = requests.get(f"https://bookateetime.teequest.com/search/{course[1]}/{date}?selectedPlayers={players}&selectedHoles=18")

  # Decode the response content as a string
  content_str = response.content.decode('utf-8')
  soup = BeautifulSoup(content_str, 'html.parser')
  tee_times = []

  for tee_time_div in soup.find_all('div', class_='tee-time'):
    tt = {
      'course': course[0],
      'tee_time': tee_time_div['data-date-time'],
      'price': float(tee_time_div['data-price']),
      'players': int(tee_time_div['data-available']),
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)

# Convert the 'date_time' column to datetime format
tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], format='%Y%m%d%H%M')

In [82]:
for course in golfback_courses.items():
  url = f"https://api.golfback.com/api/v1/courses/{course[1]}/date/{date}/teetimes"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://golfback.com/",
    "Content-Type": "application/json",
  }
  params = {
    "date": date,
    "course_id": course[1],
    "players": players
  }

  response = requests.post(url, headers=headers, json=params)
  tee_times_raw = response.json()['data']

  tee_times = []
  for tee_time in tee_times_raw:
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['dateTime'], format='%Y-%m-%dT%H:%M:%S%z').tz_localize(None),
      'price': float(tee_time['rates'][0]['price']),
      'players': tee_time['playersMax']
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


In [83]:
tee_times_df

,course,tee_time,price,players
0,Shoal Creek,2025-04-26 16:20:00,47.0,4
1,Shoal Creek,2025-04-26 16:30:00,47.0,4
2,Shoal Creek,2025-04-26 16:40:00,47.0,4
3,Shoal Creek,2025-04-26 16:50:00,47.0,4
4,Shoal Creek,2025-04-26 17:00:00,47.0,4
...,...,...,...,...
289,Royal Meadows,2025-04-26 22:48:00,32.0,4
290,Royal Meadows,2025-04-26 22:57:00,32.0,4
291,Royal Meadows,2025-04-26 23:06:00,32.0,4
292,Royal Meadows,2025-04-26 23:15:00,32.0,4


In [84]:
for course in foreup_courses.items():
  flip_date = datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y')
  url = f"https://foreupsoftware.com/index.php/api/booking/times?time=all&date={flip_date}&holes=all&players={players}&booking_class=14824&schedule_id={course[1]}&api_key=no_limits"
  headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": f"https://foreupsoftware.com/index.php/booking/22857/7340",
    "Content-Type": "application/json",
  }
  params = {
    "date": datetime.strptime(date, '%Y-%m-%d').strftime('%m-%d-%Y'),
    "players": players,
  }

  response = requests.get(url, headers=headers, json=params)
  tee_times_raw = response.json()

  tee_times = []
  for tee_time in tee_times_raw:
    tt = {
      'course': course[0],
      'tee_time': pd.to_datetime(tee_time['time'], format='%Y-%m-%d %H:%M'),
      'price': float(tee_time['green_fee'] + tee_time['cart_fee']),
      'players': tee_time['available_spots']
    }
    tee_times.append(tt)

  tee_times_df = pd.concat([tee_times_df, pd.DataFrame(tee_times)], ignore_index=True)


In [85]:
tee_times_df['tee_time'] = pd.to_datetime(tee_times_df['tee_time'], utc=True)

In [86]:
tee_times_df

,course,tee_time,price,players
0,Shoal Creek,2025-04-26 16:20:00+00:00,47.0,4
1,Shoal Creek,2025-04-26 16:30:00+00:00,47.0,4
2,Shoal Creek,2025-04-26 16:40:00+00:00,47.0,4
3,Shoal Creek,2025-04-26 16:50:00+00:00,47.0,4
4,Shoal Creek,2025-04-26 17:00:00+00:00,47.0,4
...,...,...,...,...
312,Tomahawk Hills,2025-04-26 17:10:00+00:00,33.0,4
313,Tomahawk Hills,2025-04-26 17:20:00+00:00,33.0,4
314,Tomahawk Hills,2025-04-26 17:30:00+00:00,33.0,4
315,Tomahawk Hills,2025-04-26 17:40:00+00:00,33.0,4


In [87]:
tee_times_df.course.value_counts()

course
Drumm Farm - Executive         116
Drumm Farm - Full               33
WinterStone                     29
Royal Meadows                   29
Adams Pointe                    22
Painted Hills                   21
Hodge Park                      12
Shoal Creek                     11
Tomahawk Hills                  10
Dubs Dread                       9
Heritage Park                    7
Sycamore Ridge                   7
Teetering Rocks                  6
Paradise Pointe - The Posse      5
Name: count, dtype: int64

In [90]:
tee_times_df[
  (tee_times_df['tee_time'].dt.hour >= 9) &
  (tee_times_df['tee_time'].dt.hour < 15) &
  (tee_times_df['price'] <= 90) &
  (tee_times_df['price'] >= 25)
].sort_values(by=['course', 'tee_time']).to_csv('tee_times.csv', index=False)

In [79]:
tee_times_df[
  (tee_times_df['tee_time'].dt.hour >= 16) &
  (tee_times_df['price'] >= 25) &
  (tee_times_df['players'] == 4)
].to_csv('tee_times.csv', index=False)